In [12]:
import pandas as pd
import csv

In [13]:
#read the downloaded activity detail for a ride
srcfile = './Data/750228_839792480_2017-01-21.txt'
ridesrc = pd.read_csv(filepath_or_buffer=srcfile, sep='|', quotechar='^')

In [14]:
ridesrc.head()

,AthleteID,ActivityID,StartTime,TotalElapsedTime,TotalDistanceMeters,MaxSpeedMPH,MeasuredTime,Latitude,Longitude,AltitudeMeters,DistanceMeters,CurrentSpeedMPH,CurrentGrade
0,750228,839792480,2017-01-21 10:48:23,3:35:02,48615.1,33.3,2017-01-21 10:50:32,47.663058,-122.045852,111.5,68.9,0.2,-1.2
1,750228,839792480,2017-01-21 10:48:23,3:35:02,48615.1,33.3,2017-01-21 10:50:34,47.663056,-122.045773,111.5,74.8,0.4,1.9
2,750228,839792480,2017-01-21 10:48:23,3:35:02,48615.1,33.3,2017-01-21 10:50:35,47.663056,-122.045729,111.9,78.0,0.4,1.7
3,750228,839792480,2017-01-21 10:48:23,3:35:02,48615.1,33.3,2017-01-21 10:50:36,47.663052,-122.045689,112.0,81.1,0.4,0.0
4,750228,839792480,2017-01-21 10:48:23,3:35:02,48615.1,33.3,2017-01-21 10:50:38,47.663038,-122.045616,111.8,86.8,6.7,-1.7


In [15]:
#these values are present in every row, so let's just capture them and then delete them from the DataFrame
athleteid = ridesrc.ix[0]['AthleteID']
activityid = ridesrc.ix[0]['ActivityID']
start_time = ridesrc.ix[0]['StartTime']
total_elapsed = ridesrc.ix[0]['TotalElapsedTime']
total_distance = ridesrc.ix[0]['TotalDistanceMeters']
max_speed = ridesrc.ix[0]['MaxSpeedMPH']

drop_list = ['AthleteID', 'ActivityID', 'StartTime', 'TotalElapsedTime', 'TotalDistanceMeters','MaxSpeedMPH']
ridesrc.drop(drop_list,axis=1,inplace=True)

In [16]:
#add a rolling MPH column. We will use this to create a color-coded speed map for the ride
ridesrc['AverageSpeedMPH']=ridesrc['CurrentSpeedMPH'].rolling(center=True,min_periods=1,window=5).mean().round(decimals=1)

In [17]:
ridesrc.head()

,MeasuredTime,Latitude,Longitude,AltitudeMeters,DistanceMeters,CurrentSpeedMPH,CurrentGrade,AverageSpeedMPH
0,2017-01-21 10:50:32,47.663058,-122.045852,111.5,68.9,0.2,-1.2,0.3
1,2017-01-21 10:50:34,47.663056,-122.045773,111.5,74.8,0.4,1.9,0.4
2,2017-01-21 10:50:35,47.663056,-122.045729,111.9,78.0,0.4,1.7,1.6
3,2017-01-21 10:50:36,47.663052,-122.045689,112.0,81.1,0.4,0.0,2.9
4,2017-01-21 10:50:38,47.663038,-122.045616,111.8,86.8,6.7,-1.7,3.9


In [18]:
def set_color(avg_speed):
    if avg_speed < 3.0:
        return '#a50026'
    elif avg_speed < 4.0:
        return '#d73027'
    elif avg_speed < 5.0:
        return '#f46d43'
    elif avg_speed < 6.0:
        return '#fdae61'
    elif avg_speed < 7.0:
        return '#fee090'
    elif avg_speed < 8.0:
        return '#ffffbf'
    elif avg_speed < 10.0:
        return '#e0f3f8'
    elif avg_speed < 12.0:
        return '#abd9e9'
    elif avg_speed < 15.0:
        return '#74add1'
    elif avg_speed < 18.0:
        return '#4575b4'
    else:
        return '#313695'
    
#Map colors based on www.ColorBrewer.org, by Cynthia A. Brewer, Penn State
#colorbrewer2.org

In [19]:
#apply color to avg speed based on set_color function
ridesrc['ColorMap'] = ridesrc['AverageSpeedMPH'].apply(set_color)

In [20]:
output=''
for index, row in ridesrc.iterrows():
    if output == '':
        output = output + ''.join([str(row['Latitude']),',',str(row['Longitude']),',',str(row['AverageSpeedMPH'])])
        output = output + ''.join([',',row['ColorMap']])
    else:
        output = output + ''.join(['@',str(row['Latitude']),',',str(row['Longitude']),',',str(row['AverageSpeedMPH'])])
        output = output + ''.join([',',row['ColorMap']])


In [21]:
ofile = open('./Data/map_src.txt', 'w')
ofile.write(output)
ofile.close()